In [1]:
PREDICTORS = [
    "age",
    "region",
    "gender",
]

IMPUTATIONS = [
    "employment_income",
    "pension_income",
    "self_employment_income",
    "savings_interest_income",
    "dividend_income",
    "pension_income",
    "employment_expenses",
    "property_income",
    "gift_aid",
    "pension_contributions",
][:1]

In [2]:
from survey_enhance.impute import Imputation
from datasets.spi import SPI_2019_20

spi = SPI_2019_20()

In [3]:
spi.person.gender

0          MALE
1          MALE
2          MALE
3          MALE
4          MALE
          ...  
42317    FEMALE
42323    FEMALE
43246    FEMALE
43312    FEMALE
43313    FEMALE
Name: gender, Length: 779700, dtype: object

In [4]:
from policyengine_uk import Microsimulation
import pandas as pd

simulation = Microsimulation(
    dataset=spi,
    dataset_year=2019,
)

df = pd.DataFrame(simulation.calculate_dataframe(
    PREDICTORS + IMPUTATIONS, 2022
))

In [5]:
imputation = Imputation()
imputation.train(
    df[PREDICTORS],
    df[IMPUTATIONS],
    num_trees=64,
)

Training models: 100%|██████████| 1/1 [00:01<00:00,  1.08s/it]


In [6]:
imputation.predict([[50, "SOUTH_EAST", "MALE"]], mean_quantile=0.5)

[[     0.          48661.1484375    9209.3359375   61253.2890625
   33401.18359375      0.          28812.52148438  22836.59179688
   35636.8203125   31053.49609375 170740.875      238823.796875
   10991.44335938   9209.3359375  338152.28125     39270.40234375
   77473.671875    13339.13085938  21556.03515625 105112.3515625
  124699.53125     98176.              0.         108847.3046875
   10244.45214844  14214.17773438  77009.46875     10244.45214844
  251842.78125         0.         238823.796875    43859.0625
   76134.421875    26144.6953125    9209.3359375    9209.3359375
       0.          35962.296875        0.         117384.3515625
  132324.171875     9155.97949219  20915.75585938 177143.65625
   35962.296875    46206.75       108847.3046875    9209.3359375
   50688.6953125   13339.13085938 124699.53125     87291.2734375
   22836.59179688  55133.29296875 152599.65625         0.
  129122.78125    308902.25       251842.78125    145129.734375
   77009.46875    196746.84375     1

,employment_income
0,53844.110764


In [7]:
imputation.save("spi_income_1.pkl")

In [8]:
from survey_enhance.impute import Imputation

imputation.predict([[51, "SOUTH_EAST"]])

ValueError: 3 columns passed, passed data had 2 columns